In [1]:
#import awswrangler as wr
import pandas as pd
import s3fs
import sagemaker

In [21]:
import boto3
import json

In [3]:
# Get SageMaker session & default S3 bucket
sagemaker_session = sagemaker.Session()
s3 = sagemaker_session.boto_session.resource("s3")
bucket = sagemaker_session.default_bucket()  


In [4]:
dynamodb = boto3.resource('dynamodb')

In [19]:
kinesis = boto3.client("kinesis")

In [5]:
table = dynamodb.Table('iserikov_suspicious_ips') 

In [10]:
table.put_item(Item={
            'suspicious_ips': '127.0.0.2', 'name':'igor2'
        })

{'ResponseMetadata': {'RequestId': 'KEST611F534H4CME6629LAACOJVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 04 May 2022 08:31:17 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'KEST611F534H4CME6629LAACOJVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [12]:
is_suspicious = table.get_item(Key={'suspicious_ips': '127.0.0.2'})

In [14]:
is_suspicious

{'Item': {'name': 'igor2', 'suspicious_ips': '127.0.0.2'},
 'ResponseMetadata': {'RequestId': 'TULCMJVAPQ3ST878V1U02139MJVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 04 May 2022 08:33:37 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '66',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'TULCMJVAPQ3ST878V1U02139MJVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '1639987735'},
  'RetryAttempts': 0}}

In [15]:
KINESIS_ITEM_STREAM = "iserikov-bd-events"

In [37]:
kinesis_record = {"device_id": "253896", "device_type": "mobile:ios", "item_id": "1038",
 "ts": "2021-07-16T13:52:08Z", "user_ip": "154.111.78.181"}

In [38]:
kin_records = [{"PartitionKey": 'sau', "Data": json.dumps(kinesis_record)}]

In [41]:
kinesis.put_records(StreamName=KINESIS_ITEM_STREAM, Records=kin_records)

{'FailedRecordCount': 0,
 'Records': [{'SequenceNumber': '49629220316393340472969121358268099957186839250351423490',
   'ShardId': 'shardId-000000000000'}],
 'ResponseMetadata': {'RequestId': 'd338264f-bf82-5c5c-89d5-fd0765f60de8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd338264f-bf82-5c5c-89d5-fd0765f60de8',
   'x-amz-id-2': 'GG/eQhYz1/A2x4xI/XmWwKukqMgSyR3nvpEi4oVHxQ3cwdJ5cUDiGA8FcH3IlNy4PsJVTw5zIdmpY4484SOyBe5ksUOMVNB+',
   'date': 'Thu, 05 May 2022 14:02:48 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '146'},
  'RetryAttempts': 0}}

In [53]:
import base64

In [54]:
# Sample event from Kinesis SQl Destination Stream
kinesis_dest_stream={'invocationId': '8a237820-cd14-40fd-b6ab-2b6728d29003', 'applicationArn': 'arn:aws:kinesisanalytics:us-east-1:571632058847:application/iserikov-bd-sql-app', 'records': [{'recordId': 'e136c416-a6a4-49b1-a2d5-3d7987a476a9', 'lambdaDeliveryRecordMetadata': {'retryHint': 0}, 'data': 'eyJpdGVtX2lkIjoxMDM4LCJpdGVtc19jb3VudCI6MX0='}, {'recordId': '010e26f8-a70f-49bb-9422-7c1fcbeaab84', 'lambdaDeliveryRecordMetadata': {'retryHint': 0}, 'data': 'eyJpdGVtX2lkIjoxMDM4LCJpdGVtc19jb3VudCI6Mn0='}]}

In [58]:
for record in kinesis_dest_stream['records']:
    print(record['data'])
    payload=base64.b64decode(record["data"])
    record = json.loads(payload)
    print (record)
    if record['items_count']>1:
        print ('SNS->')

eyJpdGVtX2lkIjoxMDM4LCJpdGVtc19jb3VudCI6MX0=
{'item_id': 1038, 'items_count': 1}
eyJpdGVtX2lkIjoxMDM4LCJpdGVtc19jb3VudCI6Mn0=
{'item_id': 1038, 'items_count': 2}
SNS->


In [59]:
'''try:
        client.publish(TopicArn=topic_arn, Message='Investigate sudden surge in orders', Subject='Cadabra Order Rate Alarm')
        print('Successfully delivered alarm message')
    except Exception:
        print('Delivery failure')'''

"try:\n        client.publish(TopicArn=topic_arn, Message='Investigate sudden surge in orders', Subject='Cadabra Order Rate Alarm')\n        print('Successfully delivered alarm message')\n    except Exception:\n        print('Delivery failure')"

## Write to SNS

In [65]:
topic_arn = 'arn:aws:sns:us-east-1:571632058847:iserikov-bd-alert'
client = boto3.client('sns')
try:
    message = str(record['item_id']) + ' has too many views'
    client.publish(TopicArn=topic_arn, Message='Investigate sudden surge in orders', Subject='Rate Alarm')
    print('Successfully delivered alarm message')
except Exception as e:
    print('error: '+e)

Successfully delivered alarm message


## Read from MySQl

In [70]:
import awswrangler as wr

In [72]:
#!pip install awswrangler

In [73]:
wr.__version__

'2.10.0'

In [74]:
databases = wr.catalog.databases()
print(databases)

           Database Description
0  iserikov-glue-db            


In [75]:
con_mysql = wr.mysql.connect("iserikov-mysql-connection")

In [76]:
wr.mysql.read_sql_query("SELECT * FROM tickit.desc_items_desc_csv", con=con_mysql)

,item_id,description
0,1038,TV
1,1039,Phone
